# Builder Tutorial number 7

The builder tutorials demonstrate how to build an operational GSFLOW model using `pyGSFLOW` from shapefile, DEM, and other common data sources. These tutorials focus on the `gsflow.builder` classes.

## Building PRMS parameter input files

In this tutorial, we demonstrate how to build PRMS parameter input files for GSFLOW and PRMS models. The building method relies on a number of defaults stored within pyGSFLOW, which allows the user to 1) rapidly generate a basic PRMS model, and 2) later adjust it using pyGSFLOW built in functionality.

In [1]:
import os
import utm
import shapefile
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import flopy
from gsflow.builder import GenerateFishnet, FlowAccumulation

### The `PrmsBuilder` class

The `PrmsBuilder` class builds a PRMS parameter file from pyGSFLOW default parameter values (from PRMS tables 5.2.0) and user supplied cascade routing information. 

The `PrmsBuilder` class has 4 required parameters and 3 optional parameters:

**REQUIRED parameters**
   - `stream_data_obj` : A `_StreamsObj` object that contains stream connectivity information (created by `FlowAccumulation.make_streams()`)
   - `cascades_obj` : A `_Cascades` object that contains cascade routing information (created by `FlowAccumulation.get_cascades()`)
   - `modelgrid` : modelgrid instance from GenerateFishnet (flopy.discretization.StructuredGrid object)
   - `dem` : numpy array of resampled DEM map data
   
**OPTIONAL parameters**
   - `hru_type` : optional hru_type array that defines inactive=0, land=1, lake=2, swale=3, and glacier=4 cells. 
   - `hru_subbasin` : optional hru_subbasin array that can be defined by using the `FlowAccumulation.define_subbasins()` method.
   - `defaults` : optional parameter, user can supply a gsflow.builder.Defaults instance to ModflowBuilder to use a custom set of default values
   
Let's start by importing the class

In [2]:
from gsflow.builder import PrmsBuilder

## Applying the methods to the Sagehen 50m example problem

In this example the methods are applied directly to the Sagehen 50m model as they are presented.

In [3]:
# define the input and output data paths
input_ws = os.path.join("data", "sagehen", "50m_tutorials")
output_ws = os.path.join("data", "temp")

# define the modelgrid and resampled DEM data paths
mg_file = os.path.join(input_ws, "sagehen_50m_grid.bin")
dem_data = os.path.join(input_ws, "sagehen_50m_dem.txt")

# define the watershed data path
watershed_file = os.path.join(input_ws, "sagehen_50m_watershed.txt")

# define the stream information and cascade routing binary data paths
stream_file = os.path.join(input_ws, "sagehen_50m_streams.bin")
cascade_file = os.path.join(input_ws, "sagehen_50m_cascades.bin")

Load the previously processed data

In [4]:
# load modelgrid, dem, watershed, stream information, and cascade routing file
modelgrid = GenerateFishnet.load_from_file(mg_file)
dem_data = np.genfromtxt(dem_data)
watershed = np.genfromtxt(watershed_file, dtype=int)
strm_obj = FlowAccumulation.load_streams(stream_file)
cascades = FlowAccumulation.load_cascades(cascade_file)

### Building a parameter file object

Now that the modelgrid, resampled dem, watershed, and streamflow information has been loaded. `PrmsBuilder` can be instantiated

In this example all parameters except `defaults` are supplied to `PrmsBuilder` on instantiation. The built in `Defaults` will be used to create the initial iteration of the Sagehen 50m model.

In [5]:
prmsbuild = PrmsBuilder(
    strm_obj,
    cascades,
    modelgrid,
    dem_data.ravel(),
    hru_type=watershed.ravel(),
    hru_subbasin=watershed.ravel()
)

To build a parameter file object the user just needs to run the `build()` method

In [6]:
parameters = prmsbuild.build()

print(parameters)
print(parameters.record_names[0:10])

['ndays', 'ndepl', 'nlake', 'nmonths', 'nrain', 'ntemp', 'one', 'nsegment', 'nreach', 'ngw']


## `PrmsParameters` object overview

The `PrmsParameters` object holds parameter file information and allows the user to edit parameters, add parameters, remove parameters, and write PRMS parameter information to parameter file(s). The `pygsflow_quickstart` notebooks show how to interact with this class in more detail than will be presented here. 

Instead this example shows how to update the values of two existing variables "hru_lat" and "hru_lon" and how to write the parameter object to file for later updating and tuning.

Futher tuning and calibration is presented in a later Notebook of the Builder_tutorial series.

### `ParameterRecord` explanation

All parameter values are stored in a parameter record object that contains information about the parameters data type, prms dimensions, and parameter file name path. Let's look at a `ParameterRecord` object quickly

In [7]:
ssr2gw_rate = parameters.get_record("ssr2gw_rate")

print(ssr2gw_rate)
print(type(ssr2gw_rate))
print(ssr2gw_rate.dims)
print(ssr2gw_rate.values[0:5])


####
ssr2gw_rate 10
1
nssr
20562
2
0.1
0.1
0.1
0.1.
.
.
####
<class 'gsflow.prms.prms_parameter.ParameterRecord'>
[20562]
[0.1 0.1 0.1 0.1 0.1]


### Updating all values of an existing parameter

The `PrmsParameters` object allows the user to easily update existing parameters using pyGSFLOW. Here is an example of updating the PRMS "hru_lat" and "hru_lon" parameters.

Because of python overloading methods arithmatic and setting values can be applied to the `ParameterRecord` object by calling it's name as an attribute.

In [8]:
lat, lon = utm.to_latlon(
    modelgrid.xcellcenters.ravel(),
    modelgrid.ycellcenters.ravel(),
    10,
    "N"
)

# set hru_lat and hru_lon values, using dynamic methods
parameters.hru_lat = lat
parameters.hru_lon = lon

Now let's confirm that are values where set correctly and that "hru_lon" is still a `ParameterRecord` object.

In [9]:
if not np.allclose(parameters.hru_lat.values, lat):
    raise Exception()

if not np.allclose(parameters.hru_lon.values, lon):
    raise Exception()

    
print(parameters.hru_lon)
print(type(parameters.hru_lon))


####
hru_lon 10
1
nhru
20562
2
-126.32087051684917
-126.32029025941492
-126.31971000150182
-126.31912974310997.
.
.
####
<class 'gsflow.prms.prms_parameter.ParameterRecord'>


### Writing the `PrmsParameter` object to a PRMS parameter file 

This section shows how to write the `PrmsParameter` object to a GSFLOW/PRMS parameter input file using the built in `write()` method. The Builder Tutorial series will revist this file later for further tuning and to adjust parameters that represent soil properties, vegetative cover, and imperviousness.

In [10]:
param_file = os.path.join(output_ws, "sagehen_50m_initial.param")
parameters.write(param_file)